In [2]:
import os
import sys
sys.path.append('../../')
from recs.backtest import get_backtest_answer
from recs.utils.path import *
import __init__
from recs.utils.common import predict_ranking, readjson2dict, recall_evaluate, id2cat
import recs
from tqdm import tqdm
import pandas as pd
tqdm.pandas()

In [3]:
u2idx = readjson2dict("crm_idx")
tran_data = pd.read_pickle(DATA_PATH / "trans_buy.pkl")
real = get_backtest_answer(tran_data, start=20200000, end=20210000)

100%|██████████| 28496/28496 [01:15<00:00, 379.09it/s]


In [4]:
target_date = 202000
trans_buy_train = tran_data[tran_data.buy_date < (target_date)*100]
crm = pd.read_csv(DATA_PATH / "crm_for_clustering_2019.csv")
crm = crm[crm.yyyymm == 201912]
crm['身分證字號'] = crm['身分證字號'].apply(lambda x: int(u2idx.get(x, -1)))
crm = crm[crm['身分證字號'] > -1].drop(columns='index')

# print(crm.columns)
target_users = crm[crm.yyyymm == 201912]['身分證字號'].unique().tolist()
best_sells = trans_buy_train[
    (trans_buy_train.buy_date > 20171200 )
].fund_id.value_counts().rename_axis('fund_id').reset_index(name='count')
trans_buy_train

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id_number,fund_id,buy_date,yyyymm
1,13627,930,20170327,201702
5,13651,748,20190709,201906
8,13651,2053,20170926,201708
11,13653,2518,20170616,201705
12,13654,2091,20160322,201602
...,...,...,...,...
395706,67934,3041,20180116,201712
395707,67951,2108,20191114,201910
395708,67969,482,20180509,201804
395709,67971,2058,20180515,201804


In [5]:
crm

,儲蓄型保險AUM,SNAUM,總投資理財AUM,黃金AUM,基金申購註記,當年度債券AP,外幣定存AUM,持有信用卡註記,存款AUM,投資經驗註記,...,職業_37_旅遊業,職業_38_社工/義工,職業_39_保全業,職業_41_網路拍賣,職業_42_虛擬貨幣業,職業_99_其他,職業_ND,性別_0.0,性別_1.0,性別_2.0
0,0.0,0.0,13.244581,0.000000,0,0.0,0.0,1,13.071854,0,...,0,0,0,0,0,0,0,0,1,0
2,0.0,0.0,13.124358,12.638764,0,0.0,0.0,0,14.633031,0,...,0,0,0,0,0,0,0,0,0,1
3,0.0,0.0,14.355737,0.000000,0,0.0,0.0,1,10.533806,0,...,0,0,0,0,0,0,0,0,0,1
4,0.0,0.0,0.000000,0.000000,0,0.0,0.0,1,15.279810,0,...,0,0,0,0,0,0,0,0,1,0
5,0.0,0.0,11.273798,10.382699,0,0.0,0.0,1,14.069649,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157276,0.0,0.0,0.000000,0.000000,0,0.0,0.0,0,0.000000,0,...,0,0,0,0,0,0,0,1,0,0
157277,0.0,0.0,0.000000,0.000000,0,0.0,0.0,0,0.000000,0,...,0,0,0,0,0,0,0,1,0,0
157278,0.0,0.0,0.000000,0.000000,0,0.0,0.0,0,0.000000,0,...,0,0,0,0,0,0,0,1,0,0
157279,0.0,0.0,0.000000,0.000000,0,0.0,0.0,0,0.000000,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
# valid data
tran_data = pd.read_pickle(DATA_PATH / "trans_buy.pkl")
user2020 = list(tran_data[tran_data.buy_date > 20200000].id_number.unique())
user2017 =list(tran_data[tran_data.buy_date < 20200000].id_number.unique())
new_user = list(set(user2020) - set(user2017))
print(len(new_user))
new_real = {}
for i in tqdm(real):
    if i in new_user:
        new_real[i] = real[i]

11056


100%|██████████| 28496/28496 [00:43<00:00, 655.88it/s]


In [9]:
new_users_in_2019 = list(set(target_users) & set(new_user))

new_new_real = {}
for i in new_users_in_2019:
    
    new_new_real[i] = new_real[i]
    
print(len(new_new_real))

best_sell_rec = {}
for i in new_users_in_2019:
    best_sell_rec[i] =  best_sells['fund_id'][:10]
recall_evaluate(best_sell_rec, new_new_real)

2112


100%|██████████| 2112/2112 [00:00<00:00, 10438.99it/s]


{'recall@3': 0.04273852594165095,
 'recall@5': 0.05649999595312094,
 'recall@10': 0.06894099030552822,
 'recall@all': 0.06894099030552822}

In [30]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.cluster import Birch, DBSCAN
import itertools
import numpy as np
import umap


n_cluster = 50
n_comp = 10
mx = 0
mx_ncl = 0
mx_nco = 0

def feature_selecter(udata, method, n_components):
    if method not in ["tsne", "umap"]:
        raise ValueError("feature select method not allowed")                    
    
    if method == "tsne":
        x_embedded = TSNE(n_components=n_components, init='random', random_state=47, n_jobs=30).fit_transform(udata.values)
        x_embedded = np.array(x_embedded)
    
    if method == "umap":
        x_embedded = umap.UMAP(n_components=n_components, random_state=47).fit_transform(udata.values)
        
    return x_embedded

def clustering(user_data, x_embedded, method, n_clusters=None, eps=None, min_samples=None):
    """
    quick annotation
    n_clusters for birch
    eps, min_samples for dbscan
    """
    if method not in ["birch", "dbscan"]:
        raise ValueError("clustering method not allowed")
    
    if method == "birch":
        brc = Birch(n_clusters=n_clusters)
        brc.fit(x_embedded)
        brc_x = np.array(brc.predict(x_embedded))
        user_data['cluster'] = brc_x                        
    
    if method == "dbscan":
        dbscan = DBSCAN(
            eps=eps,
            min_samples=min_samples
        ).fit(x_embedded)         
        user_data['cluster'] = dbscan.labels_
    
    return user_data

def recommender_method(tran_data, user_data):
    clusters_top10 = {}
    clusters = user_data.cluster.value_counts().index.tolist()
    for i in clusters:
        userlen = len(user_data[user_data['cluster'] == i])
        tranlen = len(tran_data[tran_data['id_number'].isin( user_data[user_data['cluster'] == i]['身分證字號'])])
        cluster_top10 = tran_data[tran_data['id_number'].isin( 
            user_data[user_data['cluster'] == i]['身分證字號']
        )]['fund_id'].value_counts().index.tolist()[:10]
        clusters_top10[i] = cluster_top10
    
    return clusters_top10
    
def backtest(user_data, x_embedded, trans_buy_train, cluster_method="dbscan", n_clusters=50, eps=0.5, min_samples=5):
    c_user_data = clustering(user_data, x_embedded, cluster_method, n_clusters, eps, min_samples)
    cluster_top10 = recommender_method(trans_buy_train, c_user_data)

    result_by_cluster = {}
    new_users_in_2019 = list(set(target_users) & set(new_user))

    for u in tqdm(new_users_in_2019):
        temp = user_data[user_data['身分證字號']==u]
        t_cluster = temp.cluster.tolist()[0]        
        result_by_cluster[u] = cluster_top10[t_cluster]

    r = recall_evaluate(result_by_cluster, new_new_real)
    print(r)
    return r
    
def findbest2(embedding_method, cluster_method, n_components=10):
    user_data = crm[crm.yyyymm == 201912]
    u_data = user_data.drop(columns=['身分證字號', 'CIFAOCODE'])
    u_data = u_data.replace(-np.inf, -1)
    u_data.fillna(0, inplace=True)        
    
    x_embedded = feature_selecter(u_data, embedding_method, n_components)
    c_user_data = clustering(user_data, x_embedded, cluster_method)
        
    cluster_top10 = recommender_method(trans_buy_train, c_user_data)
    
    result_by_cluster = {}
    for u in tqdm(target_users):
        temp = user_data[user_data['身分證字號'] == u]
        t_cluster = temp.cluster.tolist()[0]    
        result_by_cluster[u] = cluster_top10[t_cluster]

    temp_dict = {}
    for i in cluster_rec_dict:
        temp_dict[int(i)] = best_sells['fund_id'][:10]

    
    r = recall_evaluate(cluster_rec_dict, new_real)
    return r
        
    




In [7]:
crm[crm['身分證字號'].isin(new_user)]

,儲蓄型保險AUM,SNAUM,總投資理財AUM,黃金AUM,基金申購註記,當年度債券AP,外幣定存AUM,持有信用卡註記,存款AUM,投資經驗註記,...,職業_37_旅遊業,職業_38_社工/義工,職業_39_保全業,職業_41_網路拍賣,職業_42_虛擬貨幣業,職業_99_其他,職業_ND,性別_0.0,性別_1.0,性別_2.0
71142,0.000000,0.0,14.446656,0.0,0,0.0,0.000000,1,13.726159,0,...,0,0,0,0,0,0,0,0,1,0
71143,0.000000,0.0,14.539413,0.0,0,0.0,0.000000,0,11.508482,1,...,0,0,0,0,0,0,0,0,1,0
71144,0.000000,0.0,11.289782,0.0,0,0.0,0.000000,1,10.488981,0,...,0,0,0,0,0,0,0,0,1,0
72951,0.000000,0.0,13.910821,0.0,0,0.0,0.000000,0,12.349744,0,...,0,0,0,0,0,0,0,0,1,0
72968,13.815511,0.0,12.887999,0.0,0,0.0,10.971546,0,12.616151,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157135,0.000000,0.0,0.000000,0.0,0,0.0,0.000000,0,0.000000,0,...,0,0,0,0,0,0,0,1,0,0
157221,0.000000,0.0,0.000000,0.0,0,0.0,0.000000,0,0.000000,0,...,0,0,0,0,0,0,0,1,0,0
157247,0.000000,0.0,0.000000,0.0,0,0.0,0.000000,0,0.000000,0,...,0,0,0,0,0,0,0,1,0,0
157257,0.000000,0.0,0.000000,0.0,0,0.0,0.000000,0,0.000000,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
embedding_method="umap"
n_components=10

user_data = crm[crm.yyyymm == 201912]
u_data = user_data.drop(columns=['身分證字號', 'CIFAOCODE'])
u_data = u_data.replace(-np.inf, -1)
u_data.fillna(0, inplace=True)        

x_embedded = feature_selecter(u_data, embedding_method, n_components)

/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


In [ ]:
# umap dbscan default 'recall@3': 0.04323324245199246, 'recall@5': 0.05881050636022228, 'recall@10': 0.09080525340057341, 'recall@all': 0.09080525340057341
# umap birch default 'recall@3': 0.05260426783864283, 'recall@5': 0.06474051122488622, 'recall@10': 0.09673095385752385, 'recall@all': 0.09673095385752385
# umap dbscan eps=6.5 min_sample=5 'recall@3': 0.04927221216283717, 'recall@5': 0.06522885652857245, 'recall@10': 0.09996868834525836, 'recall@all': 0.09996868834525836

In [31]:
mx = 0
mx_n_components = 0
for i in range(20):
    x_embedded = feature_selecter(u_data, "umap", 5*(i+1))
    r = backtest(user_data, x_embedded, trans_buy_train, cluster_method="dbscan", eps=6.5, min_samples=5)
    if r['recall@all'] > mx:
        mx = r['recall@all']
        mx_n_components = 5*(i+1)

/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 2112/2112 [00:00<00:00, 213573.07it/s]


{'recall@3': 0.05329551813926814, 'recall@5': 0.06658785369722872, 'recall@10': 0.09102921924953926, 'recall@all': 0.09102921924953926}


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 2112/2112 [00:00<00:00, 198475.76it/s]


{'recall@3': 0.04876321595071596, 'recall@5': 0.06688079255550848, 'recall@10': 0.09803604344386344, 'recall@all': 0.09803604344386344}


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 2112/2112 [00:00<00:00, 196042.36it/s]


...

{'recall@3': 0.042985978142228154, 'recall@5': 0.05924094713157212, 'recall@10': 0.09338612160644162, 'recall@all': 0.09338612160644162}


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 2112/2112 [00:00<00:00, 194745.09it/s]


{'recall@3': 0.04283867174492175, 'recall@5': 0.060152046764262675, 'recall@10': 0.09431129596911599, 'recall@all': 0.09431129596911599}


/usr/local/lib/python3.6/dist-packages/sklearn/manifold/_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
100%|██████████| 2112/2112 [00:00<00:00, 207757.64it/s]

{'recall@3': 0.04315432831057832, 'recall@5': 0.060467703329919235, 'recall@10': 0.09360670542077544, 'recall@all': 0.09360670542077544}


In [33]:
print(mx, mx_n_components)

0.1024586173976874 25


In [24]:
# find best eps for dbscan
mx = 0
mx_eps = 0

for i in range(20):    
    r = backtest(user_data, x_embedded, trans_buy_train, cluster_method="dbscan", eps=0.5*(i+1), min_samples=5)
    if r['recall@all'] > mx:
        mx = r['recall@all']
        mx_eps = 0.5*(i+1)

100%|██████████| 2112/2112 [00:00<00:00, 196403.12it/s]


{'recall@3': 0.042759757603507614, 'recall@5': 0.057968755518471436, 'recall@10': 0.08915932987964989, 'recall@all': 0.08915932987964989}


100%|██████████| 2112/2112 [00:00<00:00, 210167.98it/s]


{'recall@3': 0.04283867174492175, 'recall@5': 0.059520733632949546, 'recall@10': 0.09025285441067442, 'recall@all': 0.09025285441067442}


100%|██████████| 2112/2112 [00:00<00:00, 190547.66it/s]


...

{'recall@3': 0.0550795414076664, 'recall@5': 0.0670010426757586, 'recall@10': 0.08696201604582292, 'recall@all': 0.08696201604582292}


100%|██████████| 2112/2112 [00:00<00:00, 215994.59it/s]


{'recall@3': 0.0550795414076664, 'recall@5': 0.0670010426757586, 'recall@10': 0.08696201604582292, 'recall@all': 0.08696201604582292}


100%|██████████| 2112/2112 [00:00<00:00, 216395.59it/s]

{'recall@3': 0.0550795414076664, 'recall@5': 0.0670010426757586, 'recall@10': 0.08696201604582292, 'recall@all': 0.08696201604582292}


In [26]:
print(mx, mx_eps)

0.09996868834525836 6.5


In [28]:
# find best min_samples for dbscan
mx = 0
mx_min_samples = 0

for i in range(10):
    r = backtest(user_data, x_embedded, trans_buy_train, cluster_method="dbscan", eps=6.5, min_samples=100*(i+1))
    if r['recall@all'] > mx:
        mx = r['recall@all']
        mx_min_samples = 100*(i+1)

100%|██████████| 2112/2112 [00:00<00:00, 208304.80it/s]


{'recall@3': 0.04927221216283717, 'recall@5': 0.06522885652857245, 'recall@10': 0.09996868834525836, 'recall@all': 0.09996868834525836}


100%|██████████| 2112/2112 [00:00<00:00, 214311.95it/s]


{'recall@3': 0.049745697011322025, 'recall@5': 0.06522885652857245, 'recall@10': 0.09949520349677352, 'recall@all': 0.09949520349677352}


100%|██████████| 2112/2112 [00:00<00:00, 203159.65it/s]


...

100%|██████████| 2112/2112 [00:00<00:00, 206181.22it/s]


{'recall@3': 0.049903525294150305, 'recall@5': 0.06522885652857245, 'recall@10': 0.099542551981622, 'recall@all': 0.099542551981622}


100%|██████████| 2112/2112 [00:00<00:00, 214873.38it/s]


{'recall@3': 0.049903525294150305, 'recall@5': 0.06428188683160274, 'recall@10': 0.09881315984597985, 'recall@all': 0.09881315984597985}


KeyboardInterrupt: 

In [18]:
backtest(user_data, x_embedded, trans_buy_train, cluster_method="dbscan", eps=1, min_samples=5)

100%|██████████| 2112/2112 [00:00<00:00, 197572.71it/s]

{'recall@3': 0.04283867174492175, 'recall@5': 0.059520733632949546, 'recall@10': 0.09025285441067442, 'recall@all': 0.09025285441067442}


In [17]:
backtest(user_data, x_embedded, trans_buy_train, cluster_method="dbscan", eps=0.5, min_samples=5)


100%|██████████| 2112/2112 [00:00<00:00, 194139.04it/s]

{'recall@3': 0.042759757603507614, 'recall@5': 0.057968755518471436, 'recall@10': 0.08915932987964989, 'recall@all': 0.08915932987964989}


In [12]:
backtest(user_data, x_embedded, trans_buy_train, cluster_method="birch")

100%|██████████| 2112/2112 [00:00<00:00, 202033.71it/s]

{'recall@3': 0.05260426783864283, 'recall@5': 0.06474051122488622, 'recall@10': 0.09673095385752385, 'recall@all': 0.09673095385752385}


In [25]:
import hdbscan

ModuleNotFoundError: No module named 'sklearn.externals.joblib'

In [1]:
embedding_method="tsne"
n_components=3

user_data = crm[crm.yyyymm == 201912]
u_data = user_data.drop(columns=['身分證字號', 'CIFAOCODE'])
u_data = u_data.replace(-np.inf, -1)
u_data.fillna(0, inplace=True)

x_embedded = feature_selecter(u_data, embedding_method, n_components)
backtest(user_data, x_embedded, trans_buy_train, cluster_method="birch")
backtest(user_data, x_embedded, trans_buy_train, cluster_method="dbscan")

NameError: name 'crm' is not defined

In [49]:
len(list(set(target_users) & set(new_user)))

2112

In [51]:
len(new_real)

11056

In [37]:
print(cluster_top10)

[995, 921, 2940, 20, 722, 202, 2732, 2128, 3197, 985]


In [48]:
# umap birch 0.01971914093398498 50 12
print(mx, mx_ncl, mx_nco)

0.01971914093398498 50 12
